## Problem Data

The following cells represent the data for available grapes and for the winery products in the form of Python dictionaries. If you look closely, these are actually dictionaries of dictionaries. 

In [1]:
GRAPES = {
    'SB11': {
        'Type': 'Cabernet',
        'Locale': 'Santa Barbara',
        'Year': 2011,
        'Acidity': 0.35,
        'Sugar': 0.12,
        'Alcohol': 13.5,
        'Quantity': 50000,
        'Price': 2.35
    },
    'SL10': {
        'Type': 'Cabernet',
        'Locale': 'San Luis Obispo',
        'Year': 2010,
        'Acidity': 0.75,
        'Sugar': 0.25,
        'Alcohol': 15.3,
        'Quantity': 60000,
        'Price': 2.60
    },
    'SL11': {
        'Type': 'Cabernet',
        'Locale': 'San Luis Obispo',
        'Year': 2011,
        'Acidity': 0.55,
        'Sugar': 0.30,
        'Alcohol': 11.5,
        'Quantity': 30000,
        'Price': 2.10        
    },
    'SB10': {
        'Type': 'Merlot',
        'Locale': 'Santa Barbara',
        'Year': 2010,
        'Acidity': 0.25,
        'Sugar': 0.08,
        'Alcohol': 15.7,
        'Quantity': 200000,
        'Price': 1.55
    }
}

WINES = {
    'SB 2011 Cab': {
        'Type': 'Cabernet',
        'Locale': 'Santa Barbara',
        'Year': 2011,
        'Acidity': 0.7,
        'Sugar': 0.2,
        'Price': 9.00
    },
    'SLO 2010 Cab': {
        'Type': 'Cabernet',
        'Locale': 'San Luis Obispo',
        'Year': 2010,
        'Acidity': 0.7,
        'Sugar': 0.2,
        'Price': 9.00
    },
    'SLO 2011 Cab': {
        'Type': 'Cabernet',
        'Locale': 'San Luis Obispo',
        'Year': 2011,
        'Acidity': 0.7,
        'Sugar': 0.2,
        'Price': 9.00        
    },
    'NV Cabernet': {
        'Type': 'Cabernet',
        'Acidity': 0.7,
        'Sugar': 0.3,
        'Price': 5.50,
    },
    'NV Merlot': {
        'Type': 'Merlot',
        'Acidity': 0.3,
        'Sugar': 1.0,
        'Price': 2.95,
    }
}

In [2]:
import pandas as pd
from IPython.display import display

display(pd.DataFrame(GRAPES).T)
display(pd.DataFrame(WINES).T)

,Acidity,Alcohol,Locale,Price,Quantity,Sugar,Type,Year
SB10,0.25,15.7,Santa Barbara,1.55,200000,0.08,Merlot,2010
SB11,0.35,13.5,Santa Barbara,2.35,50000,0.12,Cabernet,2011
SL10,0.75,15.3,San Luis Obispo,2.6,60000,0.25,Cabernet,2010
SL11,0.55,11.5,San Luis Obispo,2.1,30000,0.3,Cabernet,2011


,Acidity,Locale,Price,Sugar,Type,Year
NV Cabernet,0.7,NaN,5.5,0.3,Cabernet,NaN
NV Merlot,0.3,NaN,2.95,1,Merlot,NaN
SB 2011 Cab,0.7,Santa Barbara,9,0.2,Cabernet,2011
SLO 2010 Cab,0.7,San Luis Obispo,9,0.2,Cabernet,2010
SLO 2011 Cab,0.7,San Luis Obispo,9,0.2,Cabernet,2011


In [3]:
from pyomo.environ import *

def modelLHW():
    m = ConcreteModel()
    m.dual = Suffix(direction=Suffix.IMPORT)
    
    # to simplify indexing in subsequent expressions
    Grapes = list(GRAPES.keys())
    Wines = list(WINES.keys())

    # Decision Variables
    m.G = Var(Grapes, domain=NonNegativeReals)          # Bottles of grapes consumed
    m.W = Var(Wines, domain=NonNegativeReals)           # Bottles of wine produced
    m.X = Var(Wines, Grapes, domain=NonNegativeReals)   # Bottles of grapes assigned to wine 

    # Objective
    m.profit = Objective(
        expr = sum(m.W[w]*WINES[w]['Price'] for w in Wines) - sum(m.G[g]*GRAPES[g]['Price'] for g in Grapes),
        sense = maximize)

    # Constraints on the grapes consumed
    m.spec = ConstraintList()         # generic constraints
    m.quantity  = ConstraintList()         # limit on grape consumption

    for g in Grapes:
        m.spec.add(m.G[g] == sum(m.X[w,g] for w in Wines))
        m.quantity.add(m.G[g] <= GRAPES[g]['Quantity'])

    # Constraints on the wines produced
    for w in Wines:
        m.spec.add(m.W[w] == sum([m.X[w,g] for g in Grapes]))
        m.spec.add(sum(m.X[w,g]*GRAPES[g]['Acidity'] for g in Grapes) <= WINES[w]['Acidity']*m.W[w])
        m.spec.add(sum(m.X[w,g]*GRAPES[g]['Sugar'] for g in Grapes) <= WINES[w]['Sugar']*m.W[w])
        m.spec.add(sum(m.X[w,g]*GRAPES[g]['Alcohol'] for g in Grapes) >= 10.0*m.W[w])
        m.spec.add(sum(m.X[w,g]*GRAPES[g]['Alcohol'] for g in Grapes) <= 15.0*m.W[w])
#        m.spec.add(sum(m.X[w,g] for g in Grapes if WINES[w]['Type'] == GRAPES[g]['Type']) >= 0.75*m.W[w])
#        if 'Locale' in WINES[w].keys():
#            m.spec.add(sum(m.X[w,g] for g in Grapes if WINES[w]['Locale'] == GRAPES[g]['Locale']) >= 0.85*m.W[w])
#        if 'Year' in WINES[w].keys():
#            m.spec.add(sum(m.X[w,g] for g in Grapes if WINES[w]['Year'] == GRAPES[g]['Year']) >= 0.95*m.W[w])
            
    SolverFactory('glpk').solve(m)
    return m

m = modelLHW()
m.profit()

2413500.0

### Profitability Analysis

In [4]:
print("Revenue = ${0:12.2f}".format(sum(m.W[w]()*WINES[w]['Price'] for w in WINES.keys())))
print("   Cost = ${0:12.2f}".format(sum(m.G[g]()*GRAPES[g]['Price'] for g in GRAPES.keys())))
print(" Profit = ${0:12.2f}".format(m.profit()))

Revenue = $  3060000.00
   Cost = $   646500.00
 Profit = $  2413500.00


### Grapes Consumed

In [5]:
print('Grapes         Available        Used   Remaining')
for g in GRAPES.keys():
    print("{0:12s}".format(g), end='')
    print("{0:12.1f}".format(GRAPES[g]['Quantity']), end='')
    print("{0:12.1f}".format(m.G[g]()), end='')
    print("{0:12.1f}".format(GRAPES[g]['Quantity']-m.G[g]()))

Grapes         Available        Used   Remaining
SB11             50000.0     50000.0         0.0
SL10             60000.0     60000.0         0.0
SL11             30000.0     30000.0         0.0
SB10            200000.0    200000.0         0.0


### Blending of Grapes to form Wine Products

In [6]:
import pandas as pd
from IPython.display import display, HTML

def profitLHW(m):
    df = pd.DataFrame([[m.X[w,g]() for g in GRAPES.keys()] for w in WINES.keys()],
        index=WINES.keys(), columns = GRAPES.keys())
    df['Bottles'] = df.sum(axis=1)
    df['Cost'] = pd.Series([sum([m.X[w,g]()*GRAPES[g]['Price'] for g in GRAPES.keys()]) for w in WINES.keys()],index=WINES.keys())
    df['Revenue'] = pd.Series([WINES[w]['Price'] for w in WINES.keys()],index=WINES.keys())*df['Bottles']
    df['Profit'] = df['Revenue'] - df['Cost']
    df.loc['Total'] = df.sum()
    display(round(df,3))
    
def productLHW(m):
    df = pd.DataFrame()
    for w in WINES.keys():
        if m.W[w]() >= 0.1:
            df.loc[w,'Acidity'] = sum([m.X[w,g]()*GRAPES[g]['Acidity'] for g in GRAPES.keys()])/m.W[w]()
            df.loc[w,'Sugar'] = sum([m.X[w,g]()*GRAPES[g]['Sugar'] for g in GRAPES.keys()])/m.W[w]()
            df.loc[w,'Alcohol'] = sum([m.X[w,g]()*GRAPES[g]['Alcohol'] for g in GRAPES.keys()])/m.W[w]()
            if 'Locale' in WINES[w].keys():
                df.loc[w,'Locale'] = sum([m.X[w,g]() for g in GRAPES.keys() if WINES[w]['Locale']==GRAPES[g]['Locale']])/m.W[w]()
            if 'Vintage' in WINES[w].keys():
                df.loc[w,'Year'] = sum([m.X[w,g]() for g in GRAPES.keys() if WINES[w]['Year']==GRAPES[g]['Year']])/m.W[w]()
    display(round(df,3))
    
profitLHW(m)
productLHW(m)

,SB11,SL10,SL11,SB10,Bottles,Cost,Revenue,Profit
SB 2011 Cab,50000.0,60000.0,30000.0,200000.0,340000.0,646500.0,3060000.0,2413500.0
SLO 2010 Cab,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SLO 2011 Cab,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NV Cabernet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NV Merlot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Total,50000.0,60000.0,30000.0,200000.0,340000.0,646500.0,3060000.0,2413500.0


,Acidity,Sugar,Alcohol,Locale
SB 2011 Cab,0.379,0.135,14.935,0.735


## Question 1. Marginal Value of Additional Grapes

What is the maximum that the vintner should pay for one more bottle of the 2011 Cabernet Sauvignon harvested in Santa Barbara? Compare the result to price of the 2011 Cabernet Sauvignon Santa Barbara Wine. Explain what you see.

### Answer

The first step is to find the marginal values of adding additional bottles of grapes.

In [82]:
g = 'SB11'

# find the zero-based index of SB11 in GRAPES
idx = list(GRAPES.keys()).index(g)

# get dual value of the associated constraint.
val = m.dual[m.quantity[idx+1]]

# print result
print("The marginal value of 1 additional bottle of", g, "grapes is $", round(val,2))
print("The market price of", g, "is $", GRAPES[g]['Price'])
print("The maximum price a producer should pay for", g, "is $", round(val + GRAPES[g]['Price'],2))

The marginal value of 1 additional bottle of SB11 grapes is $ 7.04
The market price of SB11 is $ 2.35
The maximum price a producer should pay for SB11 is $ 9.39


## Question 2. Constrained Demand for Vintage Wine

What is the impact on the optimal allocation if only 50,000 bottles of vintage Cabernet Sauvignon can be sold?

### Answer

First, let's see the impact if we don't change the allocation. 

In [85]:
n = 50000

VINTAGES = [w for w in WINES.keys() if 'Year' in WINES[w].keys()]

vintage_revenue = 0
vintage_bottles = 0
for v in VINTAGES:
    vintage_revenue += m.W[v]()*WINES[v]['Price']
    vintage_bottles += m.W[v]()
    
print("With full demand:")
print(" Vintage Bottles =   ", round(vintage_bottles,0))
print(" Vintage Revenue = $", round(vintage_revenue,0))
print(" Average Price (Vintage) =      $", round(vintage_revenue/vintage_bottles,2))
print()
print("Demand limited to  ", n, "bottles:")
print("       Revenue = $", round(vintage_revenue*n/vintage_bottles,2))
print()
print("Loss assuming average price = $", round(vintage_revenue*(vintage_bottles-n)/vintage_bottles,2))

With full demand:
 Vintage Bottles =    52632.0
 Vintage Revenue = $ 473684.0
 Average Price (Vintage) =      $ 9.0

Demand limited to   50000 bottles:
       Revenue = $ 450000.0

Loss assuming average price = $ 23684.21


Now let's add a constraint that produces no more than 50,000 bottles of vintage wine.

In [86]:
m = modelLHW()
profit_before = m.profit()

profitLHW(m)
productLHW(m)

m.demand = Constraint(expr = sum([m.W[v] for v in VINTAGES]) <= 50000)
SolverFactory('glpk').solve(m)

profitLHW(m)
productLHW(m)

print("Profit with reallocated production = ", m.profit())
print("Change in Profit = ", round(m.profit() - profit_before,2))

,SB11,SL10,SL11,SB10,Bottles,Cost,Revenue,Profit
SB 2011 Cab,50000.0,0.0,0.000,2631.579,52631.579,121578.947,473684.211,352105.263
SLO 2010 Cab,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000
SLO 2011 Cab,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000
NV Cabernet,0.0,60000.0,9795.918,23265.306,93061.224,212632.653,511836.735,299204.082
NV Merlot,0.0,0.0,20204.082,101020.408,121224.490,199010.204,357612.245,158602.041
Total,50000.0,60000.0,30000.000,126917.293,266917.293,533221.805,1343133.190,809911.386


,Acidity,Sugar,Alcohol,Locale
SB 2011 Cab,0.345,0.118,13.61,1.0
NV Cabernet,0.604,0.213,15.00,NaN
NV Merlot,0.300,0.117,15.00,NaN


,SB11,SL10,SL11,SB10,Bottles,Cost,Revenue,Profit
SB 2011 Cab,47500.0,0.0,0.000,2500.000,50000.000,115500.000,450000.000,334500.000
SLO 2010 Cab,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000
SLO 2011 Cab,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000
NV Cabernet,2500.0,60000.0,8826.531,23775.510,95102.041,217262.755,523061.224,305798.469
NV Merlot,0.0,0.0,21173.469,105867.347,127040.816,208558.673,374770.408,166211.735
Total,50000.0,60000.0,30000.000,132142.857,272142.857,541321.429,1347831.633,806510.204


,Acidity,Sugar,Alcohol,Locale
SB 2011 Cab,0.345,0.118,13.61,1.0
NV Cabernet,0.596,0.209,15.00,NaN
NV Merlot,0.300,0.117,15.00,NaN


Profit with reallocated production =  806510.2040816338
Change in Profit =  -3401.18


The conclusion is that there is about a $20,000 benefit to knowing the demand constraint for vintage wines in advance of actual production.

## Question 3. Upgrading the NV Cabernet

Your marketing department tells you that reducing the sugar content of the non-varietal Cabernet from 0.3 to 0.2 would allow them to get a higher price. Is that true? How high would the price have to be to make it worthwhile? What if the new price was $6.50?

In [87]:
WINES['NV Cabernet']['Sugar'] = 0.3
WINES['NV Cabernet']['Price'] = 5.5
m1 = modelLHW()
print("Profit = ",round(m1.profit(),2))
profitLHW(m1)
productLHW(m1)

Profit =  809911.39


,SB11,SL10,SL11,SB10,Bottles,Cost,Revenue,Profit
SB 2011 Cab,50000.0,0.0,0.000,2631.579,52631.579,121578.947,473684.211,352105.263
SLO 2010 Cab,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000
SLO 2011 Cab,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000
NV Cabernet,0.0,60000.0,9795.918,23265.306,93061.224,212632.653,511836.735,299204.082
NV Merlot,0.0,0.0,20204.082,101020.408,121224.490,199010.204,357612.245,158602.041
Total,50000.0,60000.0,30000.000,126917.293,266917.293,533221.805,1343133.190,809911.386


,Acidity,Sugar,Alcohol,Locale
SB 2011 Cab,0.345,0.118,13.61,1.0
NV Cabernet,0.604,0.213,15.00,NaN
NV Merlot,0.300,0.117,15.00,NaN


In [10]:
WINES['NV Cabernet']['Sugar'] = 0.2
WINES['NV Cabernet']['Price'] = 6.5
m2 = modelLHW()
print("Profit = ",round(m2.profit(),2))
profitLHW(m2)
productLHW(m2)

Profit =  898461.31


,SB11,SL10,SL11,SB10,Bottles,Cost,Revenue,Profit
SB 2011 Cab,41709.402,0.0,0.000,2195.232,43904.633,101419.703,395141.700,293721.997
SLO 2010 Cab,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000
SLO 2011 Cab,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000
NV Cabernet,8290.598,60000.0,6581.197,24957.265,99829.060,227987.179,648888.889,420901.709
NV Merlot,0.000,0.0,23418.803,117094.017,140512.821,230675.214,414512.821,183837.607
Total,50000.000,60000.0,30000.000,144246.514,284246.514,560082.096,1458543.410,898461.314


,Acidity,Sugar,Alcohol,Viticulture,Vintage
SB 2011 Cab,0.345,0.118,13.61,1.0,0.95
NV Cabernet,0.579,0.200,15.00,NaN,NaN
NV Merlot,0.300,0.117,15.00,NaN,NaN


So yes, the winery could make substantially more profit if the dryer, non-vintage cabernet could be sold at $6.50 per bottle.